In [1]:
from dotenv import load_dotenv
import os
import requests
import json
from pydantic import BaseModel
from openai import OpenAI

In [2]:
load_dotenv()
diffbot_token = os.environ.get("DIFF_BOT_TOKEN")
print(diffbot_token)

e358a4872d18362d2334c9b6aef5b4dc


In [3]:
surl = f"https://api.diffbot.com/v3/list?url=https://habr.com/ru/news/&token={diffbot_token}"


headers = {
    "Content-Type": "application/json"
}

response1 = requests.request("GET", surl, headers=headers)

In [4]:
url2 = f"https://api.diffbot.com/v3/list?url=https://retailnews.ai/category/news&useProxy=default&token={diffbot_token}"

headers = {
    "Content-Type": "application/json"
}

response2 = requests.request("GET", url2, headers=headers)

In [5]:
data1 = json.loads(response1.text).get("objects")
# Получаем только items из первого элемента списка data
if data1 and isinstance(data1, list) and len(data1) > 0:
    items1 = data1[0].get("items", [])
else:
    items1 = []

#items1=items1[:30]

print(items1)

[{'date': 'Mon, 16 Dec 2024 13:35:06 GMT', 'summary': 'Android XR представляет собой новую операционную систему (ОС), разработанную Google в сотрудничестве с Samsung и Qualcomm. ОС предлагает расширенные возможности для работы, обучения и творчества. Платформа объединяет искусственный интеллект, дополненную (AR) и виртуальную реальность (VR), чтобы трансформировать взаимодействие с устройствами XR (extended reality), такими как гарнитуры и очки.', 'image': 'https://habrastorage.org/r/w780/getpro/habr/upload_files/14e/cd8/f34/14ecd8f348845e3d8b858afd9f8c742e.jpg', 'link': 'https://habr.com/ru/news/867026/', 'title': 'Google представила ОС Android XR для\xa0AR\\VR‑шлемов и умных очков'}, {'summary': '+2', 'link': 'https://habr.com/ru/news/867026/comments/', 'title': 'Добавить в закладки'}, {'date': 'Mon, 16 Dec 2024 13:32:59 GMT', 'summary': 'Злоумышленники начали новую волну рассылок, нацеленную на российских блогеров и менеджеров популярных каналов в мессенджерах, сообщает «Лаборатория

In [6]:
data2 = json.loads(response2.text).get("objects")
# Получаем только items из первого элемента списка data
if data2 and isinstance(data2, list) and len(data2) > 0:
    items2 = data2[0].get("items", [])
else:
    items2 = []

#items2=items2[:30]

print(items2)

[{'summary': 'Saddleback Leather E-Commerce Strategy: Balancing Innovation and Authenticity In a digital-first retail world, brands often struggle to keep customers at the heart of their e-commerce strategy. Yet Saddleback Leather, known for high-quality, handcrafted leather goods, has mastered this challenge by […]', 'image': 'https://retailnews.ai/wp-content/uploads/2024/11/SaddlebagsDualThumb.png', 'ast-excerpt-container': 'Saddleback Leather E-Commerce Strategy: Balancing Innovation and Authenticity In a digital-first retail world, brands often struggle to keep customers at the heart of their e-commerce strategy. Yet Saddleback Leather, known for high-quality, handcrafted leather goods, has mastered this challenge by […]', 'fn': 'https://retailnews.ai/author/faristarik/', 'link': 'https://retailnews.ai/saddleback-leather-ecommerce-strategy/', 'posted-by': 'Faris T', 'cat-links': 'eCommerce, Luxury Retail Trends, NEWS, PODCAST', 'title': 'How Authenticity and E-Commerce Innovation D

In [7]:
#data = list(data1)  # Создаем новый список
items = items1 + items2
results = []

class ConfirmationModel(BaseModel):
    isArticleAI: int

class TextTransformer(BaseModel):
    title:str
    summary:str
    hashtags: str

In [8]:
load_dotenv()
openai_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_key)

for item in items:
    title = item.get('title')
    summary = item.get('summary')
    
    if title and summary:
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an Artificial Intelligence subject matter expert and expert at structured data extraction. You will be given structured items and you need to make conclusion about data."},
                {"role": "user", "content": "Return in structured data '1' if this text can be classified as related to Artificial Intelligence area of knowledge: "+title +" " +summary +" and '0' if not."}
            ],
            response_format=ConfirmationModel,
        )
        
        res= completion.choices[0].message.parsed
        
        if res.isArticleAI == 1:
            completion2 = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an Artificial Intelligence subject matter expert and expert at structured data extraction. You will be given structured items and you need to make conclusion about data."},
                    {"role": "user", "content": "Add 3 most relevant hashtags and return title and summary on Russian title:"+title +" summary:" +summary}
                ],
                response_format=TextTransformer,
            )
            res2= completion2.choices[0].message.parsed
            #Add 3 (three) relevant hashtags and translate title and summary to russian language"
            item["title"]= res2.title
            item["summary"]= res2.summary
            item["hashtags"]= res2.hashtags
            results.append(item)



#list(items)
# Добавляем элементы 

In [9]:
print(results)

[{'date': 'Mon, 16 Dec 2024 13:35:06 GMT', 'summary': 'Android XR представляет собой новую операционную систему (ОС), разработанную Google в сотрудничестве с Samsung и Qualcomm. ОС предлагает расширенные возможности для работы, обучения и творчества. Платформа объединяет искусственный интеллект, дополненную (AR) и виртуальную реальность (VR), чтобы трансформировать взаимодействие с устройствами XR (extended reality), такими как гарнитуры и очки.', 'image': 'https://habrastorage.org/r/w780/getpro/habr/upload_files/14e/cd8/f34/14ecd8f348845e3d8b858afd9f8c742e.jpg', 'link': 'https://habr.com/ru/news/867026/', 'title': 'Google представила ОС Android XR для AR/VR шлемов и умных очков', 'hashtags': '#AndroidXR #AR #VR'}, {'date': 'Mon, 16 Dec 2024 13:09:05 GMT', 'summary': 'В образовательной системе процесс обучения основам информационной безопасности должен начинаться с первого класса, когда школьники получают доступ к электронной школе. Заместитель директора департамента государственной по

In [10]:
# Генерация HTML-кода
from datetime import datetime
today = datetime.now()
# Форматируем дату в нужном формате
formatted_date = today.strftime('%d-%m-%Y')
#<p>{item.date}</p>
#<p><strong>Хештеги:</strong> {item["hashtags"]}</p>
#<p>{item["date"]}</p>
html_output = ""
for item in results:
    image = item.get('image')
    hashtags = item.get('hashtags')
    summary = item.get('summary')
    title = item.get('title')
    link = item.get('link')
       
    html_output += f"""
    <div class="news-item">
        <p>{formatted_date}</p>
        <h2><a href="{link}" target="_blank">{title}</a></h2>
        <img src="{image}" alt="{title}">
        <p>{summary}</p>
        <p><strong>Хештеги:</strong> {hashtags}</p>
    </div>
    """
print(html_output)


    <div class="news-item">
        <p>16-12-2024</p>
        <h2><a href="https://habr.com/ru/news/867026/" target="_blank">Google представила ОС Android XR для AR/VR шлемов и умных очков</a></h2>
        <img src="https://habrastorage.org/r/w780/getpro/habr/upload_files/14e/cd8/f34/14ecd8f348845e3d8b858afd9f8c742e.jpg" alt="Google представила ОС Android XR для AR/VR шлемов и умных очков">
        <p>Android XR представляет собой новую операционную систему (ОС), разработанную Google в сотрудничестве с Samsung и Qualcomm. ОС предлагает расширенные возможности для работы, обучения и творчества. Платформа объединяет искусственный интеллект, дополненную (AR) и виртуальную реальность (VR), чтобы трансформировать взаимодействие с устройствами XR (extended reality), такими как гарнитуры и очки.</p>
        <p><strong>Хештеги:</strong> #AndroidXR #AR #VR</p>
    </div>
    
    <div class="news-item">
        <p>16-12-2024</p>
        <h2><a href="https://habr.com/ru/news/867010/" target="_b

In [ ]:
#with open('template.html', 'r', encoding='utf-8') as f:
 #  template_content = f.read()

In [ ]:
#full_html = '<div class="news-container">\n' + "".join(html_output) + '\n</div>'
#full_html = "".join(html_output)

In [ ]:
#final_content = template_content.replace('<!-- NEWS_PLACEHOLDER -->', full_html)

In [ ]:
# Записываем итог в newsbody.html
# with open('newsbody.html', 'w', encoding='utf-8') as f:
#     f.write(final_content)

In [14]:
from dotenv import load_dotenv
import requests
import base64
import os
 
#load_dotenv()
#github_token = os.environ.get("GITHUB_TOKEN")  # или ваш токен напрямую
# github_username = "Vadimber2"
# github_repo = "xpertnetz_framer"
# file_path = "public/static/newsbody.html"  # путь к файлу в репо
# local_file_path = "newsbody.html"  # локальный файл, который вы хотите загрузить
# 
# headers = {
#     "Authorization": f"token {github_token}",
#     "Content-Type": "application/json"
# }
# 
# # Сначала получим информацию о файле (в том числе SHA)
# url = f"https://api.github.com/repos/{github_username}/{github_repo}/contents/{file_path}"
# get_response = requests.get(url, headers=headers)
# 
# # Проверяем статус GET-запроса
# if get_response.status_code == 200:
#     file_info = get_response.json()
#     existing_sha = file_info["sha"]
# elif get_response.status_code == 404:
#     # Файл не существует, SHA указывать не нужно
#     existing_sha = None
# else:
#     print("Ошибка при получении информации о файле:")
#     print(get_response.status_code, get_response.text)
#     exit(1)
# 
# # Читаем локальный файл и кодируем в base64
# with open(local_file_path, 'rb') as f:
#     content = f.read()
# encoded = base64.b64encode(content).decode('utf-8')
# 
# data = {
#     "message": "Update newsbody.html",
#     "content": encoded
# }
# 
# # Если файл уже существует, добавляем SHA
# if existing_sha is not None:
#     data["sha"] = existing_sha
# 
# # Теперь делаем PUT-запрос для обновления/создания файла
# put_response = requests.put(url, headers=headers, json=data)
# 
# if put_response.status_code in [200, 201]:
#     print("Файл успешно обновлён или создан в репозитории.")
# else:
#     print("Ошибка при загрузке файла:")
full_html = "".join(html_output)#     print(put_response.status_code, put_response.text)
with open('template.html', 'r', encoding='utf-8') as f:
    template_content = f.read()

In [13]:
from dotenv import load_dotenv
import os
from datetime import datetime
import requests
import base64

# final_content - итоговый HTML контент для нового newsbody.html
# github_token, github_username, github_repo - ваши токен, имя пользователя и репо
# Мы хотим:
# 1. Получить старую версию newsbody.html
# 2. Переименовать её в newsbody_HH_MM_DD_MM_YYYY.html
# 3. Добавить ссылку на неё в новый final_content
# 4. Загрузить сначала архивную версию, затем обновлённый newsbody.html

load_dotenv()
github_token = os.environ.get("GITHUB_TOKEN")  # или ваш токен напрямую
github_username = "Vadimber2"
github_repo = "xpertnetz_framer"

timestamp = datetime.now()
formatted_time = timestamp.strftime("%H_%M_%d_%m_%Y")  # Часы_Минуты_День_Месяц_Год
old_file_name = "public/static/newsbody.html"
backup_file_name = f"public/static/newsbody_{formatted_time}.html"

headers = {
    "Authorization": f"token {github_token}",
    "Content-Type": "application/json"
}

url = f"https://api.github.com/repos/{github_username}/{github_repo}/contents/{old_file_name}"
get_response = requests.get(url, headers=headers)

if get_response.status_code == 200:
    file_info = get_response.json()
    existing_sha = file_info["sha"]
    old_content_base64 = file_info["content"]
    old_content = base64.b64decode(old_content_base64.encode('utf-8')).decode('utf-8')

    # Теперь у нас есть старый контент. Создадим backup файл.
    # Загрузим backup с содержимым old_content
    backup_url = f"https://api.github.com/repos/{github_username}/{github_repo}/contents/{backup_file_name}"
    backup_encoded = base64.b64encode(old_content.encode('utf-8')).decode('utf-8')
    backup_data = {
        "message": f"Backup old newsbody.html as {backup_file_name}",
        "content": backup_encoded
    }

    backup_put = requests.put(backup_url, headers=headers, json=backup_data)
    if backup_put.status_code in [200, 201]:
        print("Архивная версия успешно создана:", backup_file_name)

        # Теперь добавим ссылку на эту архивную версию в новый контент
        # Например, в конец файла можно добавить ссылку на архив:
        #archive_link = f'<p><a href="/static/newsbody_{formatted_time}.html" target="_blank">Архив новостей {formatted_time}</a></p>'
        #archive_link = f'<p><a href="/static/newsbody_{formatted_time}.html">Архив новостей {formatted_time}</a></p>'
        archive_link = f'''
            <p style="text-align:center; margin-top:20px;">
                <a href="/static/newsbody_{formatted_time}.html" 
                   style="color:#666; text-decoration:none; font-weight:bold; font-size:16px;">
                   Архив новостей {formatted_time}
                </a>
            </p>
            '''


        template_content_with_link = template_content.replace('<!-- NEWS_PLACEHOLDER -->',
                                                        f'<!-- NEWS_PLACEHOLDER -->{archive_link}')
        
        
        final_content_with_link = template_content_with_link.replace('<!-- NEWS_PLACEHOLDER -->', full_html)

        # Запишем обновлённый контент локально
        with open('newsbody.html', 'w', encoding='utf-8') as f:
            f.write(final_content_with_link)

        # Теперь загрузим новый newsbody.html
        new_file_path = "public/static/newsbody.html"
        with open('newsbody.html', 'rb') as f:
            new_content = f.read()
        new_encoded = base64.b64encode(new_content).decode('utf-8')

        # Поскольку файл уже существует, нам нужно передать sha
        new_data = {
            "message": "Update newsbody.html with new content and archive link",
            "content": new_encoded,
            "sha": existing_sha
        }

        new_put_response = requests.put(url, headers=headers, json=new_data)
        if new_put_response.status_code in [200, 201]:
            print("Файл newsbody.html успешно обновлён с ссылкой на архив.")
        else:
            print("Ошибка при обновлении newsbody.html:")
            print(new_put_response.status_code, new_put_response.text)

    else:
        print("Ошибка при создании архивного файла:")
        print(backup_put.status_code, backup_put.text)

elif get_response.status_code == 404:
    # Файл newsbody.html не существует в репо, просто создадим его
    # без архивации
    print("newsbody.html не найден, создадим новый.")

    # Добавим в новый контент только ссылку на архив, если нужно
    # (Если файла нет, то и архива нет. Можно пропустить.)
    final_content= template_content.replace('<!-- NEWS_PLACEHOLDER -->', full_html)
    with open('newsbody.html', 'w', encoding='utf-8') as f:
        f.write(final_content)

    # Заливаем новый файл
    with open('newsbody.html', 'rb') as f:
        new_content = f.read()
    new_encoded = base64.b64encode(new_content).decode('utf-8')

    create_data = {
        "message": "Create newsbody.html",
        "content": new_encoded
    }

    create_response = requests.put(url, headers=headers, json=create_data)
    if create_response.status_code in [200, 201]:
        print("Файл newsbody.html успешно создан.")
    else:
        print("Ошибка при создании newsbody.html:")
        print(create_response.status_code, create_response.text)
else:
    print("Ошибка при получении информации о newsbody.html:")
    print(get_response.status_code, get_response.text)


Архивная версия успешно создана: public/static/newsbody_17_12_16_12_2024.html
Файл newsbody.html успешно обновлён с ссылкой на архив.
